In [95]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pyfolio as pf

# Define the start and end dates for data collection
start_date = dt.datetime.now() - dt.timedelta(days=3*365)  # 3 years ago
end_date = dt.datetime.now()

# Get a list of F&O listed stocks in India
fo_stocks = ['TCS.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS', 'TATAMOTORS.NS', 'PVR.NS', 'ACC.NS']  # Example stocks, there are total 198 stocks
l = len(fo_stocks)

# Iterate over each stock
for stock in fo_stocks:
    # Get the historical OHLC data using yfinance
    data = yf.download(stock, start=start_date, end=end_date)

data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-01,1339.949951,1340.000000,1308.650024,1325.949951,1280.495728,1411786
2020-07-02,1329.800049,1340.000000,1318.250000,1328.849976,1283.296387,1008546
2020-07-03,1330.000000,1337.550049,1306.000000,1314.949951,1269.872925,1019896
2020-07-06,1317.000000,1334.949951,1307.000000,1328.000000,1282.475586,1070274
2020-07-07,1327.900024,1327.900024,1307.849976,1313.099976,1268.086426,796319


In [96]:
# Calculate the 52-week rolling returns
data['RollingReturns'] = data['Close'].pct_change(52)

# Sort the stocks based on rolling returns and select the top 5 performers
top_performers = data.sort_values('RollingReturns', ascending=False).head(l)

# top_performers = data.groupby('Symbol').tail(1).nlargest(5, 'RollingReturns')

top_performers


,Open,High,Low,Close,Adj Close,Volume,RollingReturns
Date,,,,,,,
2020-11-23,1704.000000,1757.000000,1697.000000,1725.550049,1666.397339,2654717,0.311308
2022-09-15,2770.050049,2775.850098,2683.000000,2747.699951,2747.699951,2083853,0.294894
2020-11-20,1684.000000,1713.949951,1680.000000,1686.800049,1628.975830,1245196,0.289110
2020-11-18,1694.800049,1709.800049,1672.150024,1686.300049,1628.492920,1817408,0.288235
2021-07-28,2369.000000,2397.199951,2330.250000,2393.899902,2329.350342,437170,0.275522
2022-09-14,2550.050049,2785.000000,2541.100098,2743.199951,2743.199951,4456168,0.272799
2020-12-09,1695.000000,1699.099976,1653.000000,1656.599976,1599.811035,1662078,0.270740
2022-09-20,2674.399902,2762.000000,2656.000000,2725.600098,2725.600098,1961065,0.268872


In [97]:
# Define the entry price as the close price of the most recent data point
entry_price = top_performers.iloc[-1]['Close']

# Define the stop loss as 2 times the Average True Range (ATR) on the entry price
atr = top_performers['High'] - top_performers['Low']
stop_loss = entry_price - (2 * atr)

stop_loss

Date
2020-11-23    2605.600098
2022-09-15    2539.899902
2020-11-20    2657.700195
2020-11-18    2650.300049
2021-07-28    2591.700195
2022-09-14    2237.800293
2020-12-09    2633.400146
2022-09-20    2513.600098
dtype: float64

In [98]:
# Initialize an empty DataFrame to store the strategy performance
strategy_returns = pd.DataFrame()

# Calculate the weekly returns based on rebalancing the portfolio
weekly_returns = top_performers['Close'].pct_change()

# Combine the stock returns with the strategy returns DataFrame
strategy_returns = pd.concat([strategy_returns, weekly_returns], axis=1)

strategy_returns

,Close
2020-11-23 00:00:00,NaN
2022-09-15 00:00:00,0.592362
2020-11-20 00:00:00,-0.386105
2020-11-18 00:00:00,-0.000296
2021-07-28 00:00:00,0.419617
2022-09-14 00:00:00,0.145913
2020-12-09 00:00:00,-0.396107
2022-09-20 00:00:00,0.645298


In [99]:
# Rename the columns of the strategy returns DataFrame with stock symbols
# strategy_returns.set_axis(fo_stocks, axis = 0, inplace = True)

strategy_returns['Stocks'] = fo_stocks

strategy_returns

,Close,Stocks
2020-11-23 00:00:00,NaN,TCS.NS
2022-09-15 00:00:00,0.592362,RELIANCE.NS
2020-11-20 00:00:00,-0.386105,HDFCBANK.NS
2020-11-18 00:00:00,-0.000296,INFY.NS
2021-07-28 00:00:00,0.419617,ICICIBANK.NS
2022-09-14 00:00:00,0.145913,TATAMOTORS.NS
2020-12-09 00:00:00,-0.396107,PVR.NS
2022-09-20 00:00:00,0.645298,ACC.NS


In [100]:
# Perform the backtest and analyze the strategy using pyfolio
returns = strategy_returns.dropna()

returns

,Close,Stocks
2022-09-15 00:00:00,0.592362,RELIANCE.NS
2020-11-20 00:00:00,-0.386105,HDFCBANK.NS
2020-11-18 00:00:00,-0.000296,INFY.NS
2021-07-28 00:00:00,0.419617,ICICIBANK.NS
2022-09-14 00:00:00,0.145913,TATAMOTORS.NS
2020-12-09 00:00:00,-0.396107,PVR.NS
2022-09-20 00:00:00,0.645298,ACC.NS


In [101]:
pf.create_full_tear_sheet(returns['Close'])

c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
c:\Users\annar\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\plotting.py:648: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for stat, value in perf_stats[column].iteritems():


Start date,2022-09-15
End date,2022-09-20
Total months,0
,Backtest
Annual return,1403604323.9%
Cumulative returns,58.0%
Annual volatility,686.4%
Sharpe ratio,5.35
Calmar ratio,35346747.49
Stability,0.02
Max drawdown,-39.7%


IndexError: index -1 is out of bounds for axis 0 with size 0